## Пример создания векторной БД из таблицы с примерами комментариев эксперта-эпидемиолога

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader_csv = CSVLoader(file_path="expert_comments_examples_deepseek_generated_full_data.csv")

In [6]:
data = loader_csv.load()
print(data)

[Document(metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row': 0}, page_content='The infection curve is too steep: The peak is too narrowвЂ”superspreading events would create a more prolonged peak.'), Document(metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row': 1}, page_content='The infection curve is too steep: The decline post-peak is too abrupt'), Document(metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row': 2}, page_content='The infection curve is too steep: The curve assumes uniform transmission\nNone: but heterogeneity in contact rates would flatten the peak.'), Document(metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row': 3}, page_content='The infection curve is too steep: The initial growth rate is implausibly high given baseline R0 estimates.'), Document(metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row

In [7]:
from langchain_community.document_loaders.merge import MergedDataLoader

loader_all = MergedDataLoader(loaders=[loader_csv]) #loader_web, loader_txt,]

In [9]:
docs_all = loader_all.load()
len(docs_all)

2110

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs_all)

len(texts)

2110

In [16]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

c:\Users\dinara\Desktop\NLP\NLP_labs\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dinara\.cache\huggingface\hub\models--BAAI--bge-base-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP

In [17]:
from langchain_chroma import Chroma

persist_directory = 'db'

## Here you can change the embeddings etc
embedding = bge_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [18]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Same persist directory
persist_directory = 'db'

# Use the same embedding model
embedding = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")

# Load the existing database
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

# Now you can query it
results = vectordb.similarity_search("The symmetry of the infection curve is unrealistic - the descent should be slower than the ascent.", k=3)

In [22]:
results

[Document(id='9a4f90d1-26cf-42d4-8aa2-fe9f245d88ad', metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row': 5}, page_content='The infection curve is too steep: The symmetry of the curve is unrealisticвЂ”the descent should be slower than the ascent.'),
 Document(id='6d2e2762-3aa5-4080-a93a-44fa01588e92', metadata={'source': 'expert_comments_examples_deepseek_generated_full_data.csv', 'row': 1618}, page_content='The infection curve is too steep: The curve is perfectly symmetrical\nNone: but in reality,the descent is slower than the ascent.'),
 Document(id='0270a819-c86a-40eb-8441-e8f3f7d03439', metadata={'row': 217, 'source': 'expert_comments_examples_deepseek_generated_full_data.csv'}, page_content='The infection curve is too steep: The curve is perfectly symmetrical\nNone: but real epidemics have asymmetric growth and decay phases.')]

In [23]:
sentences = [doc.page_content for doc in results]
print(sentences)

['The infection curve is too steep: The symmetry of the curve is unrealisticвЂ”the descent should be slower than the ascent.', 'The infection curve is too steep: The curve is perfectly symmetrical\nNone: but in reality,the descent is slower than the ascent.', 'The infection curve is too steep: The curve is perfectly symmetrical\nNone: but real epidemics have asymmetric growth and decay phases.']


sentences = [doc.page_content for doc in results]
print(sentences)

Как делать передачу управления между агентами: https://github.com/langchain-ai/langgraph-swarm-py 
https://www.youtube.com/watch?v=WTr6mHTw5cM
https://mirror-feeling-d80.notion.site/Understanding-Multi-agent-Handoffs-1c0808527b17809ca43bf9ef4d11a471

Как сделать управляющего агента (superviser agent): https://docs.langchain.com/oss/python/langchain/supervisor
https://docs.langchain.com/oss/python/langchain/multi-agent